# Explore the reward dataset used to improve the model's helpfulness

https://huggingface.co/datasets/lvwerra/stack-exchange-paired/viewer/lvwerra--stack-exchange-paired/train

In [2]:
%pip install torch==1.13.1 torchdata

  Using cached torch-1.13.1-cp37-cp37m-manylinux1_x86_64.whl (887.5 MB)
  Using cached torchdata-0.5.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached portalocker-2.7.0-py2.py3-none-any.whl (15 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --disable-pip-version-check -q \
    transformers==4.27.2 \
    datasets==2.9.0 \
    accelerate==0.17.0 \
    evaluate==0.4.0 \
    trl==0.4.1 \
    rouge_score==0.1.2 \
    loralib==0.1.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
sagemaker 2.145.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 6.3.0 which is incompatible.
sagemaker 2.145.0 requires PyYAML==5.4.1, but you have pyyaml 6.0 which is incompatible.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-_d2rmc45
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-_d2rmc45
  Resolved https://github.com/huggingface/peft.git to commit 2822398fbe896f25d4dac5e468624dc5fd65a51b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.3.0.dev0-py3-none-any.whl size=50481 sha256=08b108895ba2f75a32059dffe40129745283c45f67fba50cadc27a1ba9469ab3
  Stored in directory: /tmp/pip-ephem-wheel-cache-0mwm7ygz/wheels/13/73/6d/ff27a3703d8bad21d7e0c24cbd9dde5d7ae78f756405707a0c
Successfully built peft

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install git+https://github.com/lvwerra/trl.git

  Cloning https://github.com/lvwerra/trl.git to /tmp/pip-req-build-aolo92bm
  Running command git clone --filter=blob:none --quiet https://github.com/lvwerra/trl.git /tmp/pip-req-build-aolo92bm
  Resolved https://github.com/lvwerra/trl.git to commit ce37eadcfa22f2a3c25422411a586b8f593e3e6e
  Preparing metadata (setup.py) ... done
  Created wheel for trl: filename=trl-0.4.2.dev0-py3-none-any.whl size=54216 sha256=2b555964b47b4d7426866ff84f35f0c4b0a7de11a50d3c2774abb8c0cce4c6ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-imltgeip/wheels/ca/6e/f4/b183ecbed483efdcd2041a8021ce7bcb9f7b09c74bff5bb00a
Successfully built trl
  Attempting uninstall: trl
    Found existing installation: trl 0.4.1
    Uninstalling trl-0.4.1:
      Successfully uninstalled trl-0.4.1

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [6]:
# reward_modeling.py

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import evaluate
import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
)
from transformers.utils import PaddingStrategy

# Define and parse arguments.
@dataclass
class ScriptArguments:
    """
    These arguments vary depending on how many GPUs you have, what their capacity and features are, and what size model you want to train.
    """

    local_rank: Optional[int] = field(default=-1, metadata={"help": "Used for multi-gpu"})
    resume_from_checkpoint: Optional[bool] = field(
        default=False,
        metadata={"help": "If you want to resume training where it left off."},
    )
    deepspeed: Optional[str] = field(
        default=None,
        metadata={
            "help": "Path to deepspeed config if using deepspeed. You may need this if the model that you want to train doesn't fit on a single GPU."
        },
    )
    per_device_train_batch_size: Optional[int] = field(default=4)
    per_device_eval_batch_size: Optional[int] = field(default=1)
    gradient_accumulation_steps: Optional[int] = field(default=1)
    learning_rate: Optional[float] = field(default=2e-5)
    weight_decay: Optional[int] = field(default=0.001)
    model_name: Optional[str] = field(        
        #default="gpt2",
        #default="EleutherAI/gpt-neo-125m",
        default="roberta-base",
        metadata={
            "help": "The model that you want to train from the Hugging Face hub. E.g. gpt2, gpt2-xl, bert, etc."
        },
    )
    bf16: Optional[bool] = field(
        default=True,
        metadata={
            "help": "This essentially cuts the training time in half if you want to sacrifice a little precision and have a supported GPU."
        },
    )
    num_train_epochs: Optional[int] = field(
        default=1,
        metadata={"help": "The number of training epochs for the reward model."},
    )
    train_subset: Optional[int] = field(
        default=100000,
        metadata={"help": "The size of the subset of the training data to use"},
    )
    eval_subset: Optional[int] = field(
        default=50000,
        metadata={"help": "The size of the subset of the eval data to use"},
    )
    gradient_checkpointing: Optional[bool] = field(
        default=False,
        metadata={"help": "Enables gradient checkpointing."},
    )
    optim: Optional[str] = field(
        default="adamw_hf",
        metadata={"help": "Enables gradient checkpointing."},
    )
    lr_scheduler_type: Optional[str] = field(
        default="linear",
        metadata={"help": "The lr scheduler"},
    )

parser = HfArgumentParser(ScriptArguments)
script_args = parser.parse_args_into_dataclasses(return_remaining_strings=True)[0]
print(script_args)

ScriptArguments(local_rank=-1, resume_from_checkpoint=False, deepspeed=None, per_device_train_batch_size=4, per_device_eval_batch_size=1, gradient_accumulation_steps=1, learning_rate=2e-05, weight_decay=0.001, model_name='roberta-base', bf16=True, num_train_epochs=1, train_subset=100000, eval_subset=50000, gradient_checkpointing=False, optim='adamw_hf', lr_scheduler_type='linear')


In [7]:
# Load the human stack-exchange-paired dataset for tuning the reward model.
train_dataset = load_dataset("lvwerra/stack-exchange-paired", data_dir="data/reward", split="train")
if script_args.train_subset > 0:
    train_dataset = train_dataset.select(range(script_args.train_subset))

validation_dataset = load_dataset("lvwerra/stack-exchange-paired", data_dir="data/evaluation", split="train")
if script_args.eval_subset > 0:
    validation_dataset = validation_dataset.select(range(script_args.eval_subset))

Using custom data configuration lvwerra--stack-exchange-paired-72b1ccbd85cc93aa
Found cached dataset parquet (/root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Using custom data configuration lvwerra--stack-exchange-paired-b55353e62cc7a74b
Found cached dataset parquet (/root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [8]:
model_name_split = script_args.model_name.split("/")[-1]
# output_name = (
#     f"{model_name_split}_peft_stack-exchange-paired_rmts__{script_args.train_subset}_{script_args.learning_rate}"
# )


# Load the value-head model and tokenizer.
config = AutoConfig.from_pretrained(script_args.model_name)

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8, # rank
    lora_alpha=32,
    lora_dropout=0.1,
)

peft_ranking_reward_public_qanda_model_base = AutoModelForSequenceClassification.from_pretrained(
    script_args.model_name, num_labels=1, torch_dtype=torch.bfloat16
)
peft_ranking_reward_public_qanda_model = get_peft_model(peft_ranking_reward_public_qanda_model_base, peft_config)
peft_ranking_reward_public_qanda_model.print_trainable_parameters()
#peft_rl_ranking_reward_public_dataset_model.config.pad_token_id = tokenizer.eos_token_id # needed for gpt2, gpt-neo, etc
#peft_rl_ranking_reward_public_dataset_model.config.use_cache = not script_args.gradient_checkpointing

tokenizer = AutoTokenizer.from_pretrained(script_args.model_name)
# tokenizer.pad_token = tokenizer.eos_token # needed for gpt2, gpt-neo

num_proc = 24  # Can adjust to be higher if you have more processors.

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

trainable params: 1477634 || all params: 125532674 || trainable%: 1.1770911531765826


In [9]:
# Turn the dataset into pairs of post + summaries, where text_j is the preferred question + answer and text_k is the other.
# Then tokenize the dataset.
def preprocess_function(examples):
    new_examples = {
        "input_ids_j": [],
        "attention_mask_j": [],
        "input_ids_k": [],
        "attention_mask_k": [],
    }
    for question, response_j, response_k in zip(examples["question"], examples["response_j"], examples["response_k"]):
        tokenized_j = tokenizer("Question: " + question + "\n\nAnswer: " + response_j, truncation=True)
        tokenized_k = tokenizer("Question: " + question + "\n\nAnswer: " + response_k, truncation=True)

        new_examples["input_ids_j"].append(tokenized_j["input_ids"])
        new_examples["attention_mask_j"].append(tokenized_j["attention_mask"])
        new_examples["input_ids_k"].append(tokenized_k["input_ids"])
        new_examples["attention_mask_k"].append(tokenized_k["attention_mask"])

    return new_examples

In [10]:
original_columns = train_dataset.column_names

# preprocess the dataset and filter out QAs that are longer than 512
train_dataset = train_dataset.map(preprocess_function, batched=True, num_proc=num_proc, remove_columns=original_columns)
train_dataset = train_dataset.filter(lambda x: len(x["input_ids_j"]) <= 512 and len(x["input_ids_k"]) <= 512)

Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-046fb9f003dba3a8.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e7b9f85b50ddcb28.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-0f525127c1faf34c.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-a5fd041451f35713.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-2cff35df2e387a80.arrow


...

Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-01cf2024d11b841b.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-aa032f51da2cbd48.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-aad628bf72667229.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-51876f863fc87749.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c97dceec54c1c201.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-72b1ccbd85cc93aa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e07b86129f99251f.arrow


In [11]:
validation_dataset = validation_dataset.map(preprocess_function, batched=True, num_proc=num_proc, remove_columns=original_columns)
validation_dataset = validation_dataset.filter(lambda x: len(x["input_ids_j"]) <= 512 and len(x["input_ids_k"]) <= 512)

Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-0363c84cc1a434d8.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-91a452c3ac1208ee.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-b4aaa8c4330a4d62.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-6ffeb0fe4c869307.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-87e71ab6ce91496a.arrow


...

Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-a0551a4045725e6b.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-44d45e75de45473b.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-5181e2a7888550be.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-217f85848efb2350.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-a15f7f84565da551.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-b55353e62cc7a74b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-eaebc55c29ef7511.arrow


In [12]:
# We need to define a special data collator that batches the data in our j vs k format.
@dataclass
class RewardDataCollatorWithPadding:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        features_j = []
        features_k = []
        for feature in features:
            features_j.append(
                {
                    "input_ids": feature["input_ids_j"],
                    "attention_mask": feature["attention_mask_j"],
                }
            )
            features_k.append(
                {
                    "input_ids": feature["input_ids_k"],
                    "attention_mask": feature["attention_mask_k"],
                }
            )
        batch_j = self.tokenizer.pad(
            features_j,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch_k = self.tokenizer.pad(
            features_k,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch = {
            "input_ids_j": batch_j["input_ids"],
            "attention_mask_j": batch_j["attention_mask"],
            "input_ids_k": batch_k["input_ids"],
            "attention_mask_k": batch_k["attention_mask"],
            "return_loss": True,
        }
        return batch

In [15]:
# Define the metric that we'll use for validation.
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, _ = eval_pred
    # Here, predictions is rewards_j and rewards_k.
    # We want to see how much of the time rewards_j > rewards_k.
    predictions = np.argmax(predictions, axis=0)
    labels = np.zeros(predictions.shape)
    return accuracy.compute(predictions=predictions, references=labels)


class RewardTrainer(Trainer):
    # Define how to compute the reward loss. We use the InstructGPT pairwise logloss: https://arxiv.org/abs/2203.02155
    def compute_loss(self, model, inputs, return_outputs=False):
        rewards_j = model(input_ids=inputs["input_ids_j"], attention_mask=inputs["attention_mask_j"])[0]
        rewards_k = model(input_ids=inputs["input_ids_k"], attention_mask=inputs["attention_mask_k"])[0]
        loss = -nn.functional.logsigmoid(rewards_j - rewards_k).mean()
        if return_outputs:
            return loss, {"rewards_j": rewards_j, "rewards_k": rewards_k}
        return loss


peft_ranking_reward_public_qanda_checkpoint='./peft_ranking_reward_public_qanda/'
    
# Define the training args. Needs to be done before the model is loaded if you are using deepspeed.
training_args = TrainingArguments(
    output_dir=peft_ranking_reward_public_qanda_checkpoint,
    learning_rate=script_args.learning_rate,
    per_device_train_batch_size=script_args.per_device_train_batch_size,
    per_device_eval_batch_size=script_args.per_device_eval_batch_size,
    num_train_epochs=script_args.num_train_epochs,
    weight_decay=script_args.weight_decay,
#    evaluation_strategy="steps",
#   eval_steps=500,
#   save_strategy="steps",
#   save_steps=500,
    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
    gradient_checkpointing=script_args.gradient_checkpointing,
    deepspeed=script_args.deepspeed,
    local_rank=script_args.local_rank,
    remove_unused_columns=False,
    label_names=[],
    bf16=script_args.bf16,
    logging_strategy="steps",
    logging_steps=1000,
    max_steps=1000,
    optim=script_args.optim,
    lr_scheduler_type=script_args.lr_scheduler_type,
)

# Train the reward model, finally!
reward_trainer = RewardTrainer(
    model=peft_ranking_reward_public_qanda_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
    data_collator=RewardDataCollatorWithPadding(tokenizer=tokenizer, max_length=512),
)

In [16]:
reward_trainer.train(script_args.resume_from_checkpoint)

Step,Training Loss
1000,0.696700


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2365: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "


TrainOutput(global_step=1000, training_loss=0.69666015625, metrics={'train_runtime': 157.839, 'train_samples_per_second': 25.342, 'train_steps_per_second': 6.336, 'total_flos': 0.0, 'train_loss': 0.69666015625, 'epoch': 0.04})

In [17]:
print("Saving last checkpoint of the model to {}".format(peft_ranking_reward_public_qanda_checkpoint))
#peft_rl_ranking_reward_public_dataset_model.save_pretrained(peft_rl_ranking_reward_public_dataset_model_checkpoint_name)
#reward_trainer.tokenizer.save_pretrained(peft_rl_ranking_reward_public_dataset_model)
#reward_trainer.unwrap_model(reward_trainer.model).save_pretrained(peft_rl_ranking_reward_public_dataset_model) # merge?
#reward_trainer.model.save_pretrained(peft_rl_ranking_reward_public_dataset_model_checkpoint_name)

merged_model = peft_ranking_reward_public_qanda_model.merge_and_unload()
merged_model.save_pretrained(peft_ranking_reward_public_qanda_checkpoint)
tokenizer.save_pretrained(peft_ranking_reward_public_qanda_checkpoint)  #    output_name + "rl_reward_model")??

Saving last checkpoint of the model to ./peft_ranking_reward_public_qanda/


('./peft_ranking_reward_public_qanda/tokenizer_config.json',
 './peft_ranking_reward_public_qanda/special_tokens_map.json',
 './peft_ranking_reward_public_qanda/vocab.json',
 './peft_ranking_reward_public_qanda/merges.txt',
 './peft_ranking_reward_public_qanda/added_tokens.json',
 './peft_ranking_reward_public_qanda/tokenizer.json')

In [18]:
%store peft_ranking_reward_public_qanda_checkpoint

Stored 'peft_ranking_reward_public_qanda_checkpoint' (str)
